## 1. Import Dependencies

In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 2. Import Dataset

In [3]:
df = pd.read_csv('../data/tese.csv', sep='|') 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 56 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   account_id                                  50000 non-null  object 
 1   country_name                                50000 non-null  object 
 2   account_category                            50000 non-null  object 
 3   bi_account_classification                   50000 non-null  object 
 4   customer_type                               50000 non-null  object 
 5   company_name                                49995 non-null  object 
 6   domain                                      49945 non-null  object 
 7   city                                        49965 non-null  object 
 8   sic_code                                    22107 non-null  float64
 9   companydataprovider_companyname             34549 non-null  object 
 10  companydat

In [4]:
df.head()

,account_id,country_name,account_category,bi_account_classification,customer_type,company_name,domain,city,sic_code,companydataprovider_companyname,...,age_customer,number_renewals,number_total_invoices,account_value_in_eur,revenue_last_12_month,years_purchased,billings_ltm_prorated,business_model,is_subscription,is_customer_perpetual
0,3609465D-A808-EB11-95AA-005056A7C91D,Brazil,Standard,Inside Sales,Customer,Guaçu S/A Papeis e Embalagens,@guacu.com.br,estiva gerbi,NaN,NaN,...,3,3,4,822.49,215.03,8086,215.03,S,True,False
1,3657E19B-FD1A-E411-80D3-005056846F2F,Germany,Standard,Inside Sales,Customer,Stadtwerke Bad Schwalbach,@bad-schwalbach.de,Bad Schwalbach,NaN,Stadtwerke Schwalbach/Taunus,...,9,2,4,2937.39,837.22,8080,837.22,S,True,False
2,8451D3C3-FC1A-E411-80D3-005056846F2F,Italy,Standard,Inside Sales,Customer,Prussiani Engineering s.r.l.,@prussiani.com,Albano Sant'Alessandro,3559.0,PRUSSIANI ENGINEERING SPA,...,13,2,6,5884.78,1157.72,12106,1157.72,S,True,False
3,B6B0B074-213F-ED11-9DB0-000D3ADEAD35,Netherlands,Standard,Inside Sales,Customer,Kadoelenwerf Jachtservice & Haven,@hotmail.com,Amsterdam,4493.0,Jachthaven Kadoelenwerf c.v.,...,1,1,2,861.84,478.80,4045,478.80,S,True,False
4,3151AAFA-F21A-E411-80D3-005056846F2F,Switzerland,Standard,Inside Sales,Customer,Casinò Lugano SA,@casinolugano.ch,Lugano,7999.0,Casinò Lugano SA,...,12,5,7,3538.54,533.25,14134,533.25,S,True,False


## 3. Data Preprocessing

### 3.1. Add SIC Code & Description

In [5]:
df_sic_codes = pd.read_excel('../data/sic_4_digit_codes.xls')

In [6]:
df_sic_codes

,Code Value,Description
0,111,Wheat
1,112,Rice
2,115,Corn
3,116,Soybeans
4,119,"Cash grains, nec"
...,...,...
1000,9651,"Regulation, miscellaneous commercial sectors"
1001,9661,Space research and technology
1002,9711,National security
1003,9721,International affairs


In [7]:
# Merge df and df_sic_codes and add the 'description' column to df
df = df.merge(df_sic_codes, left_on='sic_code', right_on='Code Value', how='left')

In [8]:
# Rename the 'description' column to 'code_description'
df.rename(columns={'Description': 'sic_code_description'}, inplace=True)

### 3.2. Add Industry

In [10]:
# Define a dictionary to map SIC code ranges to major industry groups
major_groups = {
    range(1, 10): 'Agriculture, Forestry, and Fishing',
    range(10, 15): 'Mining',
    range(15, 18): 'Construction',
    range(20, 40): 'Manufacturing',
    range(40, 50): 'Transportation, Communications, Electric, Gas, and Sanitary Services',
    range(50, 53): 'Wholesale Trade',
    range(52, 60): 'Retail Trade',
    range(60, 68): 'Finance, Insurance, and Real Estate',
    range(70, 90): 'Services',
    range(90, 100): 'Public Administration'
}

# Map SIC code ranges to major industry group descriptions
def map_to_major_group(sic_code):
    for group_range, description in major_groups.items():
        if sic_code // 100 in group_range:
            return description
    return 'Unknown'

In [11]:
df['industry'] = df['sic_code'].apply(map_to_major_group)

In [14]:
df.head()

,account_id,country_name,account_category,bi_account_classification,customer_type,company_name,domain,city,sic_code,companydataprovider_companyname,...,account_value_in_eur,revenue_last_12_month,years_purchased,billings_ltm_prorated,business_model,is_subscription,is_customer_perpetual,Code Value,sic_code_description,industry
0,3609465D-A808-EB11-95AA-005056A7C91D,Brazil,Standard,Inside Sales,Customer,Guaçu S/A Papeis e Embalagens,@guacu.com.br,estiva gerbi,NaN,NaN,...,822.49,215.03,8086,215.03,S,True,False,NaN,NaN,Unknown
1,3657E19B-FD1A-E411-80D3-005056846F2F,Germany,Standard,Inside Sales,Customer,Stadtwerke Bad Schwalbach,@bad-schwalbach.de,Bad Schwalbach,NaN,Stadtwerke Schwalbach/Taunus,...,2937.39,837.22,8080,837.22,S,True,False,NaN,NaN,Unknown
2,8451D3C3-FC1A-E411-80D3-005056846F2F,Italy,Standard,Inside Sales,Customer,Prussiani Engineering s.r.l.,@prussiani.com,Albano Sant'Alessandro,3559.0,PRUSSIANI ENGINEERING SPA,...,5884.78,1157.72,12106,1157.72,S,True,False,3559.0,"Special industry machinery, nec",Manufacturing
3,B6B0B074-213F-ED11-9DB0-000D3ADEAD35,Netherlands,Standard,Inside Sales,Customer,Kadoelenwerf Jachtservice & Haven,@hotmail.com,Amsterdam,4493.0,Jachthaven Kadoelenwerf c.v.,...,861.84,478.80,4045,478.80,S,True,False,4493.0,Marinas,"Transportation, Communications, Electric, Gas,..."
4,3151AAFA-F21A-E411-80D3-005056846F2F,Switzerland,Standard,Inside Sales,Customer,Casinò Lugano SA,@casinolugano.ch,Lugano,7999.0,Casinò Lugano SA,...,3538.54,533.25,14134,533.25,S,True,False,7999.0,"Amusement and recreation, nec",Services


### 3.3. Cleaning

We will be working with only *non-null* values.

In [15]:
df = df.dropna()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18150 entries, 2 to 49998
Data columns (total 56 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   account_id                                  18150 non-null  object 
 1   country_name                                18150 non-null  object 
 2   account_category                            18150 non-null  object 
 3   bi_account_classification                   18150 non-null  object 
 4   customer_type                               18150 non-null  object 
 5   company_name                                18150 non-null  object 
 6   domain                                      18150 non-null  object 
 7   city                                        18150 non-null  object 
 8   sic_code                                    18150 non-null  float64
 9   companydataprovider_companyname             18150 non-null  object 
 10  companydat

In [22]:
numerical_features = ['account_category', 'bi_account_classification', 'customer_type', 'companydataprovider_employeenumber',
       'companydataprovider_revenue_in_eur', 'customer_type', 'teamviewer_location', 'major_edition',
       'number_rm_licenses', 'number_rm_malwarebytes_endpoint_protection', 'number_pilot_licenses', 'number_arassist_licenses',
       'number_frontline_licenses', 'number_tvmeeting_licenses', 'number_iot_licenses', 'number_servicecamp_licenses', 
       'total_connections_12m', 'total_endpoints_12m', 'total_devices_12m', 'age_customer', 'account_value_in_eur', 
       'billings_ltm_prorated', 'business_model', 'industry']

In [23]:
data = df[features].copy()

In [31]:
data

,account_category,bi_account_classification,customer_type,companydataprovider_employeenumber,companydataprovider_revenue_in_eur,customer_type,teamviewer_location,major_edition,number_rm_licenses,number_rm_malwarebytes_endpoint_protection,...,number_iot_licenses,number_servicecamp_licenses,total_connections_12m,total_endpoints_12m,total_devices_12m,age_customer,account_value_in_eur,billings_ltm_prorated,business_model,industry
2,Standard,Inside Sales,Customer,52.0,15379307.0,Customer,EMEA,Corporate,0,0,...,0,0,3349,611,21,13,5884.78,1157.72,S,Manufacturing
3,Standard,Inside Sales,Customer,3.0,82742.0,Customer,EMEA,Business,0,0,...,0,0,78,4,2,1,861.84,478.80,S,"Transportation, Communications, Electric, Gas,..."
4,Standard,Inside Sales,Customer,242.0,15454197.0,Customer,EMEA,Business,0,0,...,0,0,243,33,3,12,3538.54,533.25,S,Services
5,Standard,Inside Sales,Customer,7.0,830000.0,Customer,AMERICAS,Remote Access,0,0,...,0,0,944,5,7,1,311.79,152.72,S,Retail Trade
7,Standard,Inside Sales,Customer,21.0,985411.0,Customer,EMEA,Remote Access,0,0,...,0,0,0,0,0,3,475.20,118.80,S,"Transportation, Communications, Electric, Gas,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49969,Standard,Inside Sales,Customer,9.0,3111000.0,Customer,EMEA,Remote Access,0,0,...,0,0,297,7,16,0,266.42,266.42,S,"Transportation, Communications, Electric, Gas,..."
49973,Standard,Inside Sales,Customer,105.0,18900000.0,Customer,AMERICAS,Corporate,0,0,...,0,0,3512,1578,168,13,34539.78,4914.15,S,Services
49981,Standard,Inside Sales,Customer,7.0,930779.0,Customer,EMEA,Business,0,0,...,0,0,561,9,2,2,1109.12,391.52,S,Construction
49994,Standard,Inside Sales,Customer,5.0,954000.0,Customer,AMERICAS,Corporate,0,0,...,0,0,775,89,22,10,28771.99,4683.26,S,Services


### 3.4. Scaling and Encoding Features

In [26]:
data.dtypes

account_category                               object
bi_account_classification                      object
customer_type                                  object
companydataprovider_employeenumber            float64
companydataprovider_revenue_in_eur            float64
customer_type                                  object
teamviewer_location                            object
major_edition                                  object
number_rm_licenses                              int64
number_rm_malwarebytes_endpoint_protection      int64
number_pilot_licenses                           int64
number_arassist_licenses                        int64
number_frontline_licenses                       int64
number_tvmeeting_licenses                       int64
number_iot_licenses                             int64
number_servicecamp_licenses                     int64
total_connections_12m                           int64
total_endpoints_12m                             int64
total_devices_12m           

In [28]:
# Separate numerical features
numerical_features = data.select_dtypes(include=['float64', 'int64'])

# Display the separated features
print("Numerical Features:")
numerical_features

Numerical Features:


,companydataprovider_employeenumber,companydataprovider_revenue_in_eur,number_rm_licenses,number_rm_malwarebytes_endpoint_protection,number_pilot_licenses,number_arassist_licenses,number_frontline_licenses,number_tvmeeting_licenses,number_iot_licenses,number_servicecamp_licenses,total_connections_12m,total_endpoints_12m,total_devices_12m,age_customer,account_value_in_eur,billings_ltm_prorated
2,52.0,15379307.0,0,0,1,1,0,0,0,0,3349,611,21,13,5884.78,1157.72
3,3.0,82742.0,0,0,0,0,0,0,0,0,78,4,2,1,861.84,478.80
4,242.0,15454197.0,0,0,0,0,0,0,0,0,243,33,3,12,3538.54,533.25
5,7.0,830000.0,0,0,0,0,0,0,0,0,944,5,7,1,311.79,152.72
7,21.0,985411.0,0,0,0,0,0,0,0,0,0,0,0,3,475.20,118.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49969,9.0,3111000.0,0,0,0,0,0,0,0,0,297,7,16,0,266.42,266.42
49973,105.0,18900000.0,0,0,0,0,0,0,0,0,3512,1578,168,13,34539.78,4914.15
49981,7.0,930779.0,0,0,0,0,0,0,0,0,561,9,2,2,1109.12,391.52
49994,5.0,954000.0,0,0,0,0,0,0,0,0,775,89,22,10,28771.99,4683.26


In [29]:
# Separate categorical features
categorical_features = data.select_dtypes(include=['object'])

print("\nCategorical Features:")
categorical_features


Categorical Features:


,account_category,bi_account_classification,customer_type,customer_type,teamviewer_location,major_edition,business_model,industry
2,Standard,Inside Sales,Customer,Customer,EMEA,Corporate,S,Manufacturing
3,Standard,Inside Sales,Customer,Customer,EMEA,Business,S,"Transportation, Communications, Electric, Gas,..."
4,Standard,Inside Sales,Customer,Customer,EMEA,Business,S,Services
5,Standard,Inside Sales,Customer,Customer,AMERICAS,Remote Access,S,Retail Trade
7,Standard,Inside Sales,Customer,Customer,EMEA,Remote Access,S,"Transportation, Communications, Electric, Gas,..."
...,...,...,...,...,...,...,...,...
49969,Standard,Inside Sales,Customer,Customer,EMEA,Remote Access,S,"Transportation, Communications, Electric, Gas,..."
49973,Standard,Inside Sales,Customer,Customer,AMERICAS,Corporate,S,Services
49981,Standard,Inside Sales,Customer,Customer,EMEA,Business,S,Construction
49994,Standard,Inside Sales,Customer,Customer,AMERICAS,Corporate,S,Services


#### 3.4.1. Scaling numerical features

In [32]:
scaler = StandardScaler()
numerical_features_scaled = pd.DataFrame(scaler.fit_transform(numerical_features), columns=numerical_features.columns)

#### 3.4.2. One-Hot encoding categorical features

In [33]:
# Process categorical features (e.g., one-hot encoding)
categorical_features_encoded = pd.get_dummies(categorical_features, drop_first=True)

### 3.5. Processed dataset

In [35]:
# Join the processed features back together
processed_df = pd.concat([numerical_features_scaled, categorical_features_encoded], axis=1)

print("Processed DataFrame:")
processed_df

Processed DataFrame:


,companydataprovider_employeenumber,companydataprovider_revenue_in_eur,number_rm_licenses,number_rm_malwarebytes_endpoint_protection,number_pilot_licenses,number_arassist_licenses,number_frontline_licenses,number_tvmeeting_licenses,number_iot_licenses,number_servicecamp_licenses,...,major_edition_Tensor,industry_Construction,"industry_Finance, Insurance, and Real Estate",industry_Manufacturing,industry_Mining,industry_Public Administration,industry_Retail Trade,industry_Services,"industry_Transportation, Communications, Electric, Gas, and Sanitary Services",industry_Wholesale Trade
0,-0.040151,-0.043930,-0.135561,-0.041636,7.649624,17.975181,0.0,-0.054903,-0.025988,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.053600,-0.057874,-0.135561,-0.041636,-0.118556,-0.055632,0.0,-0.054903,-0.025988,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.011996,-0.043861,-0.135561,-0.041636,-0.118556,-0.055632,0.0,-0.054903,-0.025988,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.052502,-0.057192,-0.135561,-0.041636,-0.118556,-0.055632,0.0,-0.054903,-0.025988,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.048659,-0.057051,-0.135561,-0.041636,-0.118556,-0.055632,0.0,-0.054903,-0.025988,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
49981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
